In [2]:
import torch
import torchvision
from torch import nn, optim
from visdom import Visdom

In [8]:
tr = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

mnist_data = torchvision.datasets.MNIST("mnist_data", download=True, transform=tr)
test_data = torchvision.datasets.MNIST("test_data", download=True, train=False, transform=tr)

dataloader = torch.utils.data.DataLoader(mnist_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

In [24]:
class NN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3)
        self.conv2 = nn.Conv2d(3, 2, 5)
        self.linear = nn.Linear(968, 10)
        
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self,X):
        n = X.size(0)
        
        X = self.relu(self.conv1(X))
        X = self.relu(self.conv2(X))
        
        X = X.view(n,-1)
        
        X = self.softmax(self.linear(X))
        return X
        

In [25]:
model = NN1()
loss_fn = nn.CrossEntropyLoss()
params = model.parameters()
optimizer = optim.SGD(params = params,lr=.01, momentum=0.9)

n_epochs = 15

for e in range(n_epochs):
    running_loss = 0
    for i,(images,labels) in enumerate(dataloader):
        
        #forward pass
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, labels)
        
        #backward pass
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(dataloader)))

Epoch 1 - Training loss: 0.4821868597635074
Epoch 2 - Training loss: 0.27203985952190374
Epoch 3 - Training loss: 0.22377058559023877
Epoch 4 - Training loss: 0.13694503497399851
Epoch 5 - Training loss: 0.11260661968975258
Epoch 6 - Training loss: 0.09852610721981633
Epoch 7 - Training loss: 0.08997314648508135
Epoch 8 - Training loss: 0.08489816069259429
Epoch 9 - Training loss: 0.08188228887855324
Epoch 10 - Training loss: 0.07757666719847046
Epoch 11 - Training loss: 0.07288251309566247
Epoch 12 - Training loss: 0.07183551359479862
Epoch 13 - Training loss: 0.06859535505738991
Epoch 14 - Training loss: 0.06457667275424252
Epoch 15 - Training loss: 0.06297226411241816


In [28]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    img = images[i].view(1, 1, 28, 28)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9742
